In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

In [2]:
citycsv = r'C:\Users\stave\OneDrive\Desktop\Bootcamp\week 6\Challenge\WeatherPy\city_data.csv'
city_csv = pd.read_csv(citycsv, index_col=0)
city_data = pd.DataFrame(city_csv).transpose()
city_data = city_data.dropna()
print(city_data)


              City  Latitude  Longitude Max Temperature: Humidity Cloudiness  \
0      yellowknife    62.456  -114.3525           264.44       79         75   
1          labuhan   -6.8844   112.2051           298.85       83        100   
2    san cristobal    7.7669    -72.225           298.96       63         61   
3        carnarvon  -24.8667   113.6333           291.59       60         99   
4            vaini     -21.2     -175.2           298.27       71         76   
..             ...       ...        ...              ...      ...        ...   
539        ipixuna   40.1292   124.3947           280.49       95         41   
540         hirara     65.75     150.85           247.47       99        100   
541         sydney  -11.9472   -75.2829           288.16       61         96   
542           miri   14.4012    33.5199           298.77       18          0   
543  ust-kamchatsk   14.6937   -17.4441           302.64       73         82   

    Wind Speed Country        Date  
0 

In [3]:
gmaps.configure(api_key=gkey)

In [4]:
locations = city_data[["Latitude", "Longitude"]].astype(float)
humidity = city_data["Humidity"]
fig = gmaps.figure()
city_marker = gmaps.symbol_layer(locations,fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2)
fig.add_layer(city_marker)
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
ideal = city_data[city_data["Humidity"].astype(int) <50]
ideal2 = ideal[ideal['Cloudiness'].astype(int) <50]

print(ideal2)  

                City  Latitude  Longitude Max Temperature: Humidity  \
28          hermanus   20.5022   -10.0711           296.92       14   
54      norman wells   29.0331    21.5482           293.16       25   
80    saint-philippe   16.1525    36.2032           301.24       48   
82     palabuhanratu  -20.4597    -66.825           294.71       12   
107  verkhnevilyuysk    -31.95   141.4333           286.56       47   
..               ...       ...        ...              ...      ...   
529    batagay-alyta   70.2396    22.3479           277.25       40   
530          eskasem   29.2747    82.1838           276.69       49   
531        belyy yar   41.1415   -73.3579           281.79       46   
536          aksarka   34.1478  -119.1951           300.07       17   
542             miri   14.4012    33.5199           298.77       18   

    Cloudiness Wind Speed Country        Date  
28           0        3.9      MR  1668715220  
54           0       4.89      LY  1668715235  
80 

In [13]:
hotel = ideal2[['City', 'Country','Latitude', 'Longitude', 'Humidity' ]].copy()
hotel['Hotel Name'] = ""
print(hotel)

                City Country  Latitude  Longitude Humidity Hotel Name
28          hermanus      MR   20.5022   -10.0711       14           
54      norman wells      LY   29.0331    21.5482       25           
80    saint-philippe      SD   16.1525    36.2032       48           
82     palabuhanratu      BO  -20.4597    -66.825       12           
107  verkhnevilyuysk      AU    -31.95   141.4333       47           
..               ...     ...       ...        ...      ...        ...
529    batagay-alyta      NO   70.2396    22.3479       40           
530          eskasem      NP   29.2747    82.1838       49           
531        belyy yar      US   41.1415   -73.3579       46           
536          aksarka      US   34.1478  -119.1951       17           
542             miri      SD   14.4012    33.5199       18           

[64 rows x 6 columns]


In [23]:
params = {
    "radius": 10000,
    "types": "hotel",
    "keyword": "hotel",
    "key": gkey
}

for index, row in ideal2.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    params["location"] = f"{lat},{lng}"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

hotel_address = requests.get(base_url, params=params)

hotel_address = hotel_address.json()

try:
    ideal2.loc[index, 'Hotel Name'] = hotel_address['results'][0]['name']
except (KeyError, IndexError):
    print('Missing')
ideal3 = ideal2.dropna()
ideal3

C:\Users\stave\AppData\Local\Temp\ipykernel_15688\4215870431.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal2.loc[index, 'Hotel Name'] = hotel_address['results'][0]['name']


,City,Latitude,Longitude,Max Temperature:,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
542,miri,14.4012,33.5199,298.77,18,0,6.4,SD,1668715513,International Hotel


In [29]:
ideal_location = ideal2[["Latitude", "Longitude"]].astype(float)
humidity = ideal2["Humidity"]
fig2 = gmaps.figure()
ideal_marker = gmaps.symbol_layer(ideal_location ,fill_color='red',
    stroke_color='red')
fig2.add_layer(ideal_marker)
fig2

Figure(layout=FigureLayout(height='420px'))

In [48]:
hotel_location = ideal3[["Latitude", "Longitude"]].astype(float)
humidity = ideal3["Humidity"]
hover_text = 'Latitude:14.4012, Longitude:33.5199, City:miri, Humidity:18,  Hotel Name: International Hotel, Country:SD '

hotel_marker = gmaps.symbol_layer(hotel_location ,fill_color='green',
    stroke_color='green', hover_text=hover_text)
fig3.add_layer(hotel_marker)
fig3

Figure(layout=FigureLayout(height='420px'))